In [1]:
import csv
import datetime
import pysolr
import spacy
from spacy import displacy

In [2]:
nlp = spacy.load("/veld/input/m3/model/model-best/")

/usr/local/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'de_pipeline' (0.0.0) was trained with spaCy v3.6 and may not be 100% compatible with the current version (3.5.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def fetch(solr_core, pagination_limit=999999):
    solr_core_url = "http://linkedcat-solr.acdh-cluster-2.arz.oeaw.ac.at/solr/" + solr_core
    solr = pysolr.Solr(solr_core_url, always_commit=True)
    solr_data = list(solr.search("*:*", rows=pagination_limit))
    solr_id_set = set(sd["id"] for sd in solr_data)
    assert len(solr_data) == len(solr_id_set)
    print(f"length solr_data: {len(solr_data)}")
    return solr_data

In [31]:
def parse_persist(solr_data, break_step, csv_file_name):
    csv_file_path = f"/veld/output/{csv_file_name}.csv"
    with open(csv_file_path, "w") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(("id", "start_char", "end_char", "text", "label"))
        data_tmp = []
        for i, entry in enumerate(solr_data):
            # verify
            entry_id = entry["id"]
            text_good = entry["ocrtext_good"]
            assert len(text_good) == 1
            text_good = text_good[0]
            text = entry["ocrtext"]
            assert len(text) == 1
            text = text[0]
            if text_good != text:
                print(f"different texts at {entry_id}")
            # parse
            doc = nlp(text_good)
            for ent in doc.ents:
                assert text_good[ent.start_char:ent.end_char] == ent.text
                data_tmp.append((entry_id, ent.start_char, ent.end_char, ent.text, ent.label_))
            # persist
            if i != 0 and (i % break_step == 0 or i == len(solr_data) - 1):
                print(f"persisting at index: {i}, current id: {entry_id}, {datetime.datetime.now()}")
                for data_row in data_tmp:
                    csv_writer.writerow(data_row)
                data_tmp = []

In [37]:
solr_data2 = fetch("linkedcat2")

length solr_data: 8051


In [ ]:
#for i, sd in enumerate(solr_data2):
#    print(i, sd["id"])

In [ ]:
parse_persist(solr_data2, 50, "linkedcat2")

In [ ]:
solr_data1 = fetch("linkedcat")

In [ ]:
parse_persist(solr_data1, "linkedcat")